### Link : https://colab.research.google.com/drive/1va8dt332N1CUXvQk6YJTQ0qM29m5I-w2?usp=sharing

In [2]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

In [3]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

In [5]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [6]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [8]:
text = "Quote: Our doubts are traitors,"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Our doubts are traitors, and nothing but fools doth venture.

The above quote is from the play, <em>The Tempest


In [9]:
os.environ["WANDB_DISABLED"] = "false"

In [10]:
lora_config = LoraConfig(
    r = 8, #RANK
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"], # Layers which we want to fine tune
    task_type = "CAUSAL_LM", # Type of Tank
)

In [11]:
data = load_dataset("b-mc2/sql-create-context")
data

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['context', 'answer', 'question'],
        num_rows: 78577
    })
})

In [12]:
data["train"][0]

{'context': 'CREATE TABLE head (age INTEGER)',
 'answer': 'SELECT COUNT(*) FROM head WHERE age > 56',
 'question': 'How many heads of the departments are older than 56 ?'}

In [13]:
data = data.map(lambda samples: tokenizer(samples["question"], samples["context"]), batched=True)
data

Map:   0%|          | 0/78577 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['context', 'answer', 'question', 'input_ids', 'attention_mask'],
        num_rows: 78577
    })
})

In [14]:
data["train"][0]

{'context': 'CREATE TABLE head (age INTEGER)',
 'answer': 'SELECT COUNT(*) FROM head WHERE age > 56',
 'question': 'How many heads of the departments are older than 56 ?',
 'input_ids': [2,
  2299,
  1767,
  13914,
  576,
  573,
  23645,
  708,
  9964,
  1178,
  235248,
  235308,
  235318,
  1654,
  2,
  32389,
  16448,
  2206,
  591,
  732,
  78910,
  235275],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [15]:
def formatting_func(example):
    text = f"Question: {example['question'][0]}\nContext: {example['context'][0]}\nAnswer: {example['answer'][0]}"
    return [text]

In [16]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=75,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/78577 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [17]:
trainer.train()

Step,Training Loss
1,3.025700
2,2.705400
3,2.866200
4,2.797000
5,2.641600
6,2.324900
7,2.214000
8,2.206100
9,2.086200
10,1.948100


TrainOutput(global_step=75, training_loss=1.207518483797709, metrics={'train_runtime': 190.3498, 'train_samples_per_second': 6.304, 'train_steps_per_second': 0.394, 'total_flos': 1252338972487680.0, 'train_loss': 1.207518483797709, 'epoch': 15.0})

In [18]:
text = """Question: What is the average number of working horses of farms with greater than 45 total number of horses?
Context: CREATE TABLE farm (Working_Horses INTEGER, Total_Horses INTEGER)"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Question: What is the average number of working horses of farms with greater than 45 total number of horses?
Context: CREATE TABLE farm (Working_Horses INTEGER, Total_Horses INTEGER)
Answer: SELECT AVG(Working_Horses) FROM farm WHERE Total_Horses > 45


In [23]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Directory where the model and tokenizer will be saved in Google Drive
output_dir_drive = "/content/drive/MyDrive/SQLModel"

# Save the model and tokenizer
model.save_pretrained(output_dir_drive)
tokenizer.save_pretrained(output_dir_drive)

print(f"Model and tokenizer saved to {output_dir_drive}")

Model and tokenizer saved to /content/drive/MyDrive/SQLModel


In [24]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Directory path in Google Drive where the model and tokenizer are saved
output_dir_drive = "/content/drive/MyDrive/SQLModel"

# Load the tokenizer and model from Google Drive
tokenizer = AutoTokenizer.from_pretrained(output_dir_drive)
model = AutoModelForCausalLM.from_pretrained(output_dir_drive)

# Example inference
text = """Question: What is the average number of working horses of farms with greater than 45 total number of horses?
Context: CREATE TABLE farm (Working_Horses INTEGER, Total_Horses INTEGER)"""

inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=20)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

`low_cpu_mem_usage` was None, now set to True since model is quantized.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Question: What is the average number of working horses of farms with greater than 45 total number of horses?
Context: CREATE TABLE farm (Working_Horses INTEGER, Total_Horses INTEGER) emphat increa increa increa increa increa increa increa increa increa increa increa increa increa increa increa increa increa increa increa
